In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
sys.path.append('/home/nadavg/mo_utils')
sys.executable


'/home/nadavg/anaconda3/envs/qdiff/bin/python'

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '7,5,3'

In [4]:
#from mo_utils.utils.torch_utils import torch_to_pt,create_torch_script
import netron

In [5]:
from omegaconf import OmegaConf
from txt2img import load_model_from_config
from ldm.models.diffusion.plms import PLMSSampler

/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/transformers/modeling_utils.py:371: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly 

In [6]:
from ldm.modules.diffusionmodules.util import normalization,GroupNorm32
import torch.nn.functional as F
#n = normalization(1280)
import torch

In [7]:
n = normalization(1280)
n(torch.randn(1,1280,4,4)).shape

torch.Size([1, 1280, 4, 4])

In [42]:
config = OmegaConf.load('/home/nadavg/q-diffusion/configs/stable-diffusion/v1-inference.yaml')
model = load_model_from_config(config, "/fastdata/users/nadavg/sd/qdiff/sd-v1-4.ckpt")

sampler = PLMSSampler(model)

/home/nadavg/q-diffusion/scripts/txt2img.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pl_sd = torch.load(ckpt, map_location="cpu")


LatentDiffusion: Running in eps-prediction mode


/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/nadavg/anaconda3/envs/qdiff/lib/python3.8/site-packages/transformers/modeling_utils.py:371: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly 

x.shape = torch.Size([3, 4, 64, 64])

t.shape = torch.Size([3]) 

c.shape = torch.Size([3, 77, 768])

In [43]:
input_shapes = [[3, 4, 64, 64],[3],[3,77,768]]

In [44]:
type(model.model.diffusion_model)

ldm.modules.diffusionmodules.openaimodel.UNetModel

In [45]:
type(model)

ldm.models.diffusion.ddpm.LatentDiffusion

In [46]:
unet = model.model.diffusion_model

In [47]:
len(unet.input_blocks),len(unet.middle_block),len(unet.output_blocks)

(12, 3, 12)

In [4]:
import torch
torch.cuda.is_available()
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

GPU 0: NVIDIA RTX 6000 Ada Generation
GPU 1: NVIDIA RTX 6000 Ada Generation
GPU 2: NVIDIA RTX 6000 Ada Generation


In [49]:
torch.device('cpu')

device(type='cpu')

In [51]:
unet.multi_gpu = True

In [67]:
device0= torch.device('cuda:0')
device1= torch.device('cuda:1')

In [53]:
unet= unet.to('cuda')

In [54]:
lt = lambda l : str(type(l)).split('.')[-1].split('\'')[0]

In [56]:
in_b =[]
for i,inb_i in enumerate(unet.input_blocks):
    in_b.append([])
    for l in inb_i:
        in_b[i].append(lt(l))
in_b
        

[['Conv2d'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['Downsample'],
 ['ResBlock'],
 ['ResBlock']]

In [57]:
in_b=[]
for l in unet.middle_block:
    in_b.append(lt(l))
in_b

['ResBlock', 'SpatialTransformer', 'ResBlock']

In [58]:
in_b =[]
for i,inb_i in enumerate(unet.output_blocks):
    in_b.append([])
    for l in inb_i:
        in_b[i].append(lt(l))
in_b
        

[['ResBlock'],
 ['ResBlock'],
 ['ResBlock', 'Upsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer', 'Upsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer', 'Upsample'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer'],
 ['ResBlock', 'SpatialTransformer']]

In [59]:
type(unet.output_blocks[1][0])

ldm.modules.diffusionmodules.openaimodel.ResBlock

In [60]:
unet.output_blocks[5][0].updown

False

In [61]:
c2d = unet.input_blocks[0][0]
rb = unet.input_blocks[1][0]
us = unet.output_blocks[2][1]
ds = unet.input_blocks[3][0]
st = unet.input_blocks[1][1]
type(c2d),type(rb),type(ds),type(us),type(st)

(torch.nn.modules.conv.Conv2d,
 ldm.modules.diffusionmodules.openaimodel.ResBlock,
 ldm.modules.diffusionmodules.openaimodel.Downsample,
 ldm.modules.diffusionmodules.openaimodel.Upsample,
 ldm.modules.attention.SpatialTransformer)

In [66]:
next(us.parameters()).device

device(type='cuda', index=1)

In [20]:
ds.use_conv,us.use_conv

(True, True)

In [21]:
type(rb.in_layers[0])

ldm.modules.diffusionmodules.util.GroupNorm32

In [22]:
rb.emb_layers

Sequential(
  (0): SiLU()
  (1): Linear(in_features=1280, out_features=320, bias=True)
)

In [23]:
rb.out_layers

Sequential(
  (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
  (1): SiLU()
  (2): Dropout(p=0, inplace=False)
  (3): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [24]:
rb.skip_connection

Identity()

In [25]:
st.norm,st.proj_in,st.proj_out

(GroupNorm(32, 320, eps=1e-06, affine=True),
 Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1)),
 Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1)))

In [26]:
len(st.transformer_blocks)

1

In [42]:
st.transformer_blocks[0].attn1, st.transformer_blocks[0].attn2

(CrossAttention(
   (to_q): Linear(in_features=320, out_features=320, bias=False)
   (to_k): Linear(in_features=320, out_features=320, bias=False)
   (to_v): Linear(in_features=320, out_features=320, bias=False)
   (qk_matmul): CrossQKMatMul()
   (smv_matmul): CrossSMVMatMul()
   (to_out): Sequential(
     (0): Linear(in_features=320, out_features=320, bias=True)
     (1): Dropout(p=0.0, inplace=False)
   )
 ),
 CrossAttention(
   (to_q): Linear(in_features=320, out_features=320, bias=False)
   (to_k): Linear(in_features=768, out_features=320, bias=False)
   (to_v): Linear(in_features=768, out_features=320, bias=False)
   (qk_matmul): CrossQKMatMul()
   (smv_matmul): CrossSMVMatMul()
   (to_out): Sequential(
     (0): Linear(in_features=320, out_features=320, bias=True)
     (1): Dropout(p=0.0, inplace=False)
   )
 ))

In [43]:
inb_i = unet.input_blocks[0]
#len(inb_i)
for l in inb_i:
    print(type(l))

<class 'torch.nn.modules.conv.Conv2d'>


In [44]:
lt(l)

'Conv2d'

In [45]:
rb

ResBlock(
  (in_layers): Sequential(
    (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
    (1): SiLU()
    (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (h_upd): Identity()
  (x_upd): Identity()
  (emb_layers): Sequential(
    (0): SiLU()
    (1): Linear(in_features=1280, out_features=320, bias=True)
  )
  (out_layers): Sequential(
    (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
    (1): SiLU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (skip_connection): Identity()
)

In [46]:
model.model.diffusion_model

UNetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=320, out_features=1280, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=1280, out_features=320, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0.0, inplace=False)
          (3): Conv2d(320, 320, kernel_size=(3

In [71]:
import torch
device0
device=device0
input_data = [torch.randn(input_shape).to(device) for input_shape in input_shapes]
[input_data[0].shape,input_data[1].shape,input_data[2].shape]
#input_data


[torch.Size([3, 4, 64, 64]), torch.Size([3]), torch.Size([3, 77, 768])]

In [72]:
input_data = tuple(input_data)

In [73]:
input_data[2].device

device(type='cuda', index=0)

In [50]:
_=model.cpu()

In [74]:
input_unet = {'x':input_data[0],'timesteps':input_data[1],'context':input_data[2]}

In [75]:
out_unet = unet(**input_unet)

In [76]:
out_unet.device

device(type='cuda', index=1)

In [52]:
in_rs1 = c2d(input_data[0])

In [53]:
from torchviz import make_dot

In [54]:
from ldm.modules.diffusionmodules.util import timestep_embedding
t_emb = timestep_embedding(input_data[1], model.model.diffusion_model.model_channels, repeat_only=False)
emb = model.model.diffusion_model.time_embed(t_emb)

In [55]:
emb.shape

torch.Size([3, 1280])

In [56]:
in_rs1 = c2d(input_data[0])

In [57]:
rs1_out= rb(in_rs1,emb)

In [38]:
#make_dot(rs1_out, params=dict(list(rb.named_parameters()))).render("rb1", format="svg")
#make_dot(rs1_out, params=None).render("rb1", format="svg")



In [37]:
_=rb.eval()

In [38]:
type(rb)
rb.use_checkpoint = False

In [40]:
rb.use_checkpoint

False

In [58]:
from ldm.modules.diffusionmodules.openaimodel import ResBlock


In [62]:
rb.use_conv

False

In [63]:
rb_new = ResBlock(rb.channels,rb.emb_channels,dropout=0.0,use_checkpoint=False)


In [64]:
#traced_resblock = torch.jit.script(rb_new)#._forward,example_inputs=[rb,in_rs1,emb])
#traced_resblock.save('script_resblock.pt')

In [71]:
rb_new.eval()
rb_new.requires_grad_(False)
traced_resblock = torch.jit.trace(rb_new,(in_rs1,emb),_store_inputs=True)
traced_resblock.save('traced_resblock2.pt')

RuntimeError: _Map_base::at

In [45]:
with torch.no_grad():
    torch.onnx.export(rb._forward,(in_rs1,emb),'resblock.onnx',
                   input_names=['x','timestep_embedding'], output_names=['res_block_out'])

AttributeError: 'function' object has no attribute 'modules'

In [90]:
model.model.diffusion_model.model_channels

320

In [88]:
yhat = model.model.diffusion_model(*input_data)

In [96]:
make_dot(yhat, params=dict(list(model.model.diffusion_model.named_parameters()))).render("rnn_torchviz", format="png")


'rnn_torchviz.png'

In [89]:
model.model.diffusion_model

UNetModel(
  (time_embed): Sequential(
    (0): Linear(in_features=320, out_features=1280, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=1280, out_features=320, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0, inplace=False)
          (3): Conv2d(320, 320, kernel_size=(3, 

In [83]:
import hiddenlayer as hl

transforms = [ hl.transforms.Prune('Constant') ] #

In [91]:
graph = hl.build_graph(model.model.diffusion_model, input_data, transforms=transforms)
graph.theme = hl.graph.THEMES['blue'].copy()
graph.save('rnn_hiddenlayer', format='png')

IndexError: Argument passed to at() was not in the map.

In [56]:
#outout_unet= model.model.diffusion_model(**input_unet)
outout_unet= model.model.diffusion_model(*input_data)

outout_unet.shape

torch.Size([3, 4, 64, 64])

In [79]:
input_names = ['x','timesteps','context']
output_names = ['h']
torch.onnx.export(model.model.diffusion_model,(input_data[0],input_data[1],input_data[2]),
                   'UNetModel.onnx',
                   input_names=input_names, output_names=output_names)

RuntimeError: _Map_base::at

In [90]:
torch.onnx._optimize_trace

AttributeError: module 'torch.onnx' has no attribute '_optimize_trace'

In [75]:
traced_model = torch.jit.trace(model.model.diffusion_model, (input_data[0],input_data[1],input_data[2]))
#traced_model = torch.jit.trace(model.model.diffusion_model, input_data)

#traced_model = torch.jit.trace(model.model.diffusion_model, **input_unet)



RuntimeError: _Map_base::at

In [14]:
create_torch_script(model.model.diffusion_model,input_shapes)

RuntimeError: _Map_base::at